In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('dodo').getOrCreate()

In [5]:
df = spark.read.csv('hdfs://localhost:9000/Python/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [7]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in df.head(2)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
                           ,outputCol= 'features')

In [16]:
output = assembler.transform(df)

In [17]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean|  502.1287775638194|
| stddev|  79.31094684054433|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean|  492.8084487385588|
| stddev|  79.20319580340706|
|    min| 302.18954780965197|
|    max|  700.9170916173961|
+-------+-------------------+



In [24]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [25]:
lr_model = lr.fit(train_data)

In [26]:
test_results = lr_model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.93626303724659|
|   10.9138668868992|
|  5.056402441439445|
| -12.81111225960666|
|-21.538422622742132|
| 19.326976058084085|
| 0.8707459300492246|
| -7.546882958671972|
| 18.256119788621163|
|  7.042835360768038|
| 2.0655791864119806|
|  -3.82004792497537|
| -2.405181518567531|
| -9.042991289181543|
|  8.274557664698989|
| -8.841652831936415|
|-17.109151949752516|
| 11.255599435894169|
|-13.658444858655912|
|  8.304155865159032|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

10.040342498429245

In [29]:
test_results.r2

0.9838230243079202

In [30]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.7377203726281...|
|[31.0472221394875...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.3123495994443...|
|[31.3895854806643...|
|[31.4474464941278...|
|[31.6005122003032...|
|[31.6548096756927...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8854062999117...|
|[31.9048571310136...|
|[31.9096268275227...|
|[31.9365486184489...|
|[31.9549038566348...|
+--------------------+
only showing top 20 rows



In [33]:
prediction = lr_model.transform(unlabeled_data)

In [34]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.7040880353809|
|[30.7377203726281...| 450.8668753093307|
|[31.0472221394875...|387.44099674758195|
|[31.0662181616375...|  461.744405467281|
|[31.1239743499119...| 508.4854764625079|
|[31.3123495994443...|444.26444196985653|
|[31.3895854806643...|409.19886512993367|
|[31.4474464941278...|  426.149625053896|
|[31.6005122003032...| 460.9167317024758|
|[31.6548096756927...| 468.2205883667805|
|[31.7366356860502...|494.86786706911994|
|[31.8124825597242...| 396.6303929087726|
|[31.8186165667690...| 448.8238548887032|
|[31.8279790554652...|449.04573883612306|
|[31.8512531286083...| 464.7176890020994|
|[31.8854062999117...| 398.9449258044119|
|[31.9048571310136...|491.05900937256865|
|[31.9096268275227...|  552.190436237345|
|[31.9365486184489...| 440.8578297539841|
|[31.9549038566348...|431.69372407476794|
+--------------------+------------